## Data Load

In [3]:
import pandas as pd

df = pd.read_csv("dataset/synthetic_logs.csv")
df.head(10)

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,bert
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,bert
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,bert
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,bert


In [4]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [5]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [6]:
df[df.target_label=='System Notification'].sample(10)

,timestamp,source,log_message,target_label,complexity
1226,4/15/2025 22:34,ModernHR,File data_8435.csv uploaded successfully by us...,System Notification,regex
1497,9/18/2025 20:17,AnalyticsEngine,File data_2691.csv uploaded successfully by us...,System Notification,regex
2256,12/26/2025 14:07,ThirdPartyAPI,File data_6957.csv uploaded successfully by us...,System Notification,regex
1656,12/10/2025 10:23,ModernHR,Backup completed successfully.,System Notification,regex
1187,7/3/2025 3:50,ModernHR,System updated to version 3.3.6.,System Notification,regex
1274,8/9/2025 17:23,ModernCRM,Backup ended at 2025-10-17 19:00:39.,System Notification,regex
1946,4/1/2025 15:27,ThirdPartyAPI,Backup started at 2025-02-07 07:50:02.,System Notification,regex
1707,4/17/2025 14:07,BillingSystem,Backup completed successfully.,System Notification,regex
325,6/2/2025 12:18,ThirdPartyAPI,Backup ended at 2025-07-18 17:06:54.,System Notification,regex
1148,8/13/2025 10:57,ThirdPartyAPI,Backup ended at 2025-08-22 12:30:23.,System Notification,regex


In [8]:
df[df.log_message.str.startswith("System reboot initiated by user")].sample(10)

,timestamp,source,log_message,target_label,complexity
2253,10/7/2025 2:20,ModernHR,System reboot initiated by user User644.,System Notification,regex
988,9/11/2025 22:23,BillingSystem,System reboot initiated by user User427.,System Notification,regex
36,11/19/2025 13:14,BillingSystem,System reboot initiated by user User243.,System Notification,regex
307,4/12/2025 0:41,BillingSystem,System reboot initiated by user User929.,System Notification,regex
2003,6/23/2025 17:54,ModernCRM,System reboot initiated by user User517.,System Notification,regex
889,11/30/2025 13:45,ModernHR,System reboot initiated by user User766.,System Notification,regex
2246,3/2/2025 22:56,ModernHR,System reboot initiated by user User488.,System Notification,regex
552,9/22/2025 20:54,ModernHR,System reboot initiated by user User421.,System Notification,regex
1803,8/22/2025 6:30,AnalyticsEngine,System reboot initiated by user User204.,System Notification,regex
1562,8/18/2025 4:17,ThirdPartyAPI,System reboot initiated by user User648.,System Notification,regex


## Clustering

In [9]:
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer

c:\Users\sudes\miniconda3\envs\log\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight embedding model

# Create embeddings
embeddings = model.encode(df['log_message'].tolist())

In [11]:
# View first 5 embeddings
embeddings[:5]

array([[-0.10293962,  0.03354594, -0.02202607, ...,  0.00457793,
        -0.04259717,  0.00322621],
       [ 0.00804572, -0.03573923,  0.04938739, ...,  0.01538319,
        -0.06230947, -0.02774666],
       [-0.00908224,  0.13003924, -0.05275568, ...,  0.02014104,
        -0.05117098, -0.02930294],
       [-0.09751046,  0.04911299, -0.03977424, ...,  0.02477502,
        -0.03546079, -0.00018598],
       [-0.10468338,  0.05926038, -0.02488499, ...,  0.02502055,
        -0.037193  , -0.0256891 ]], dtype=float32)

In [13]:
#  maximum distance of 0.2 (eps) between two points for them to be considered in the same cluster
# least 1 point (min_samples) to form a dense region

clustering = DBSCAN(eps=0.2, min_samples=1, metric='cosine').fit(embeddings)

In [14]:
# df['cluster'] = clustering.labels_: Assigns the cluster labels predicted by the model to a new column cluster in the dataframe df.
df['cluster'] = clustering.labels_

In [15]:
df.head()

,timestamp,source,log_message,target_label,complexity,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0


In [17]:
df.cluster.unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135], dtype=int64)

In [19]:
df[df.cluster==1].sample(10)

,timestamp,source,log_message,target_label,complexity,cluster
1453,9/11/2025 1:18,BillingSystem,Mail delivery error impacted email service,Critical Error,bert,1
2312,3/9/2025 21:16,ModernHR,Email service faced a delivery obstacle,Error,bert,1
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
1359,8/18/2025 15:58,ModernCRM,Email system down due to sending error,Critical Error,bert,1
1427,10/26/2025 20:25,ModernHR,Email provider had trouble sending emails,Error,bert,1
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1
2284,10/4/2025 18:15,BillingSystem,Email service down due to transmission issue,Critical Error,bert,1
1011,2/9/2025 17:04,AnalyticsEngine,Error in email sending impacted service,Critical Error,bert,1
1082,5/17/2025 11:20,ModernHR,Delivery failure impacted email services,Critical Error,bert,1
816,7/9/2025 9:38,BillingSystem,Error in email delivery affected service,Critical Error,bert,1


In [ ]:
# Groups the log messages by cluster and applies the list function to each group, resulting in a Series where each value is a list of log messages for a particular cluster.
clusters = df.groupby('cluster')['log_message'].apply(list)

# Sorts the clusters by the length of the log message lists (i.e., the number of log messages in each cluster) in descending order. The lambda function maps each list to its length,
sorted_clusters = clusters.sort_values(key=lambda x: x.map(len), ascending=False)

In [22]:
# prints the first 5 log messages from clusters that contain more than 10 messages.
print("Clustered Patterns:")
for cluster_id, messages in sorted_clusters.items():
    if len(messages) > 10:
        print(f"Cluster {cluster_id}:")
        for msg in messages[:5]:
            print(f"  {msg}")

Clustered Patterns:
Cluster 0:
  nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118
  nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180
  nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1874 time: 0.2280791
  nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2

## Classification Stage 1: Regex

In [23]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message):
            return label
    return None

In [24]:
classify_with_regex("User User123 logged in.")

'User Action'

In [25]:
classify_with_regex("System reboot initiated by user User179.")

'System Notification'

In [26]:
classify_with_regex("Hey you, chill bro")

In [29]:
# Apply regex classification and store the resulting labels in a new column
df['regex_label'] = df['log_message'].apply(lambda x: classify_with_regex(x))

In [30]:
df[df['regex_label'].notnull()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,4,System Notification
14,1/4/2025 1:43,ThirdPartyAPI,File data_3847.csv uploaded successfully by us...,System Notification,regex,4,System Notification
15,5/1/2025 9:41,ModernCRM,Backup completed successfully.,System Notification,regex,8,System Notification
18,2/22/2025 17:49,ModernCRM,Account with ID 5351 created by User634.,User Action,regex,9,User Action
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,11,User Action
...,...,...,...,...,...,...,...
2376,6/27/2025 8:47,ModernCRM,System updated to version 2.0.5.,System Notification,regex,21,System Notification
2381,9/5/2025 6:39,ThirdPartyAPI,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2394,4/3/2025 13:13,ModernHR,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2395,5/2/2025 14:29,ThirdPartyAPI,Backup ended at 2025-05-06 11:23:16.,System Notification,regex,13,System Notification


In [31]:
df[df['regex_label'].isnull()].head(5)

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None


## Classification Stage 2: Classification Using Embeddings

In [32]:
# creates a new DataFrame df_non_regex that contains only the rows from df where the regex_label column is null
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex.shape

(1910, 7)

# source = legacyCRM has very less training data

In [33]:
df_legacy = df_non_regex[df_non_regex.source=="LegacyCRM"]
df_legacy

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
60,2025-10-06 16:55:23,LegacyCRM,Lead conversion failed for prospect ID 7842 du...,Workflow Error,llm,24,None
255,2025-05-03 16:55:35,LegacyCRM,API endpoint 'getCustomerDetails' is deprecate...,Deprecation Warning,llm,48,None
377,2025-06-24 12:16:29,LegacyCRM,Customer follow-up process for lead ID 5621 fa...,Workflow Error,llm,62,None
1325,2025-04-17 07:33:44,LegacyCRM,Escalation rule execution failed for ticket ID...,Workflow Error,llm,105,None
1734,2025-04-30 07:47:30,LegacyCRM,The 'ExportToCSV' feature is outdated. Please ...,Deprecation Warning,llm,118,None
1826,2025-01-23 10:33:36,LegacyCRM,Support for legacy authentication methods will...,Deprecation Warning,llm,122,None
2217,2025-05-12 09:46:54,LegacyCRM,Task assignment for TeamID 3425 could not comp...,Workflow Error,llm,133,None


In [34]:
df_non_legacy = df_non_regex[df_non_regex.source!="LegacyCRM"]
df_non_legacy

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1,None


In [35]:
df_non_legacy.shape

(1903, 7)

### generating embedding for trainable data

In [36]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight embedding model
embeddings_filtered = model.encode(df_non_legacy['log_message'].tolist())


In [37]:
len(embeddings_filtered)

1903

### using logisticRegression on embeddings generated

In [38]:
X = embeddings_filtered
y = df_non_legacy['target_label'].values

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



In [40]:
# saves the trained Logistic Regression classifier (clf) to a joblib file

In [46]:
import os
import joblib

dir_path = 'models'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

joblib.dump(clf, f'{dir_path}/log_classifier.joblib')

['models/log_classifier.joblib']

In [47]:
# moving models file out of training folder to build app